# Photomosaic library advanced code to build mosaic using pool of images

### Code includes image scaling, partitioning, and color matching optimization
### Source Code Documentation: http://danielballan.github.io/photomosaic/docs/

In [1]:
import photomosaic as pm
from skimage.io import imread
image = imread('images/video_games_poster.jpg')

In [2]:
# Analyze the collection (the "pool") of images.
pool = pm.make_pool('tiles/*')

analyzing pool: 100%|██████████| 2600/2600 [00:09<00:00, 266.57it/s]


In [3]:
from skimage import img_as_float
image = img_as_float(image)

In [4]:
# Use perceptually uniform colorspace for all analysis.
converted_img = pm.perceptual(image)

In [5]:
# Adapt the color palette of the image to resemble the palette of the pool.
adapted_img = pm.adapt_to_pool(converted_img, pool)

In [9]:
scaled_img = pm.rescale_commensurate(adapted_img, grid_dims=(180, 270))

/Users/ianwomack/anaconda3/envs/py36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [10]:
tiles = pm.partition(scaled_img, grid_dims=(180, 270))

partitioning: depth 0: 100%|██████████| 48600/48600 [00:00<00:00, 172626.88it/s]


In [11]:
annotated_img = pm.draw_tile_layout(pm.rgb(scaled_img), tiles)

100%|██████████| 48600/48600 [00:04<00:00, 11476.16it/s]


In [12]:
import numpy

In [13]:
# Reshape the 3D array (height, width, color_channels) into
# a 2D array (num_pixels, color_channels) and average over the pixels.
tile_colors = [numpy.mean(scaled_img[tile].reshape(-1, 3), 0)
               for tile in tiles]

In [14]:
# Match a pool image to each tile.
match = pm.simple_matcher(pool)
matches = [match(tc) for tc in tile_colors]

In [15]:
# Create canvas
canvas = numpy.ones_like(scaled_img)  # white canvas


In [17]:
# Draw mosaic
mos = pm.draw_mosaic(canvas, tiles, matches)

drawing mosaic:   0%|          | 0/48600 [00:00<?, ?it/s]/Users/ianwomack/anaconda3/envs/py36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
drawing mosaic: 100%|██████████| 48600/48600 [00:06<00:00, 6966.32it/s]


In [18]:
# save mosaic
from skimage.io import imsave
imsave('mosaics/mosaic_poster_a.png', mos)

/Users/ianwomack/anaconda3/envs/py36/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
